# Q2 ML Models

Like training datasets, ML Models in EOTDL are categorized into different [quality levels](https://eotdl.com/docs/datasets/quality), which in turn will impact the range of functionality that will be available for each model.

In this tutorial you will learn about Q2 models, models with STAC metadata and the ML-Model extension (models with STAC metadata but not ML-Model extension will be qualified as Q1). 

## STAC Spec

For Q2 ML Models we rely on the [ML-Model](https://github.com/crim-ca/mlm-extension) STAC extension. Here we develop the required metadata for the [RoadSegmentation](https://www.eotdl.com/models/RoadSegmentation) Q0 model on EOTDL.

In [1]:
import eotdl

eotdl.__version__

'2024.05.02'

In [2]:
from eotdl.models import download_model

try:
	path = download_model('RoadSegmentation', path="data", version=2)
except Exception as e:
	print(e)
	path = 'data/RoadSegmentation/v2'

path

model `RoadSegmentation v2` already exists at data/RoadSegmentation/v2. To force download, use force=True or -f in the CLI.


'data/RoadSegmentation/v2'

In [3]:
import os 

os.listdir(path)

['README.md', 'model.onnx']

Our goal is to provide STAC metadata to run `model.onnx` on any inference processor that implements the ML-Model STAC extension. From the official repo:

> The STAC Machine Learning Model (MLM) Extension provides a standard set of fields to describe machine learning models trained on overhead imagery and enable running model inference.
>
> The main objectives of the extension are:
>
> 1. to enable building model collections that can be searched alongside associated STAC datasets
> 2. record all necessary bands, parameters, modeling artifact locations, and high-level processing steps to deploy an inference service.
>
>Specifically, this extension records the following information to make ML models searchable and reusable:
>
> 1. Sensor band specifications
> 2. Model input transforms including resize and normalization
> 3. Model output shape, data type, and its semantic interpretation
> 4. An optional, flexible description of the runtime environment to be able to run the model
> 5. Scientific references

Let's start with a generic `catalog` for our model.

In [14]:
import pystac

# current directory + 'data/RoadSegmentation/STAC'
root_href = os.path.join(os.getcwd(), 'data/RoadSegmentation/STAC')

catalog = pystac.Catalog(id='RoadSegmentationQ2', description='Catalog for the Road Segmentation Q2 ML Model')

Now let's create a `collection` for our model.

In [15]:
import pystac
from datetime import datetime

# Create a new Collection
collection = pystac.Collection(
    id='model',
    description='Collection for the Road Segmentation Q2 ML Model',
    extent=pystac.Extent(
        spatial=pystac.SpatialExtent([[-180, -90, 180, 90]]), # dummy extent
        temporal=pystac.TemporalExtent([[datetime(2020, 1, 1), None]]) # dummy extent
    ),
	# extra_fields={
    #     'stac_extensions': ['https://crim-ca.github.io/mlm-extension/v1.2.0/schema.json']
    # }
)

# Add the Collection to the Catalog
catalog.add_child(collection)

<Link rel=child target=<Collection id=model>>

And finally, an `item` to describe the model itself with the extension.

In [16]:
# Create a new Item
item = pystac.Item(
    id='model',
    geometry={ # dummy geometry
        "type": "Point",
        "coordinates": [125.6, 10.1]
    },
    bbox=[125.6, 10.1, 125.6, 10.1], # dummy bbox
    datetime=datetime.utcnow(), # dummy datetime
    properties={ 
		"mlm:name": "model.onnx", # name of the asset ? otherwise, how can we know which asset to use ?
		"mlm:framework": "ONNX",  # only framework support for now
		"mlm:architecture": "U-Net",
		"mlm:tasks": ["segmentation"], # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#task-enum
		"mlm:input": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#model-input-object
			"name": "RGB statellite image (HR)",
			"bands": [
				"red",
				"green",
				"blue"
			],
			"input": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#input-structure-object
				"shape": [
					-1,
					3,
					-1, # should be divisble by 16
					-1 # should be divisble by 16
				],
				"dim_order": [
					"batch",
					"channel",
					"height",
					"width"
				],
				"data_type": "float32",
                # we should add here the resize to nearest divisible by 16
				# "pre_processing_function": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#processing-expression
				# 	"format": 
				# 	"expression": 
				# }
				"description": "Model trained with 1024x1024 RGB HR images, but can work with other dimensions as long as they are divisible by 16"
			}
		},
		"mlm:output": {
			"name": "road binary mask",
			"tasks": ["segmentation"], # redundant ?
			"result": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#result-structure-object
				"shape": [-1, -1, -1],
				"dim_order": [
					"batch",
					"height",
					"width"
				],
				"data_type": "uint8",
				"description": "Binary mask of the road segmentation. 1 for road, 0 for background",
				# "post_processing_function": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#processing-expression
				# }
			},
		},
	}, 
    stac_extensions=['https://crim-ca.github.io/mlm-extension/v1.2.0/schema.json']
)

# Add the Item to the Collection
collection.add_item(item)

# Save the Catalog to a file
# catalog.normalize_and_save(root_href=root_href, catalog_type=pystac.CatalogType.SELF_CONTAINED)


<Link rel=item target=<Item id=model>>

The model weights are added as an asset to the item

In [17]:
# Create an Asset
model_asset = pystac.Asset(
    href=os.path.abspath('data/RoadSegmentation/v2/model.onnx'), 
)

# Add the Asset to the Item
item.add_asset('model', model_asset)

Finally, we validate and save the metadata

In [18]:
# Validate the Catalog

# catalog.validate_all()

catalog.normalize_and_save(root_href=root_href, catalog_type=pystac.CatalogType.SELF_CONTAINED)
